## Importing required libraries

In [ ]:
import urllib.request
import pandas as pd
import re
import time
import numpy as np
from tqdm import tqdm

## Custom functions needed

In [ ]:
def url_get_contents(url):

    req = urllib.request.Request(url=url, headers={'User-Agent': 'Mozilla/5.0'})
    f = urllib.request.urlopen(req)
    return f.read()

In [ ]:
def replace_all(dict, str):
    for key in dict:
        str = str.replace(key, dict[key])
    return str

In [ ]:
def decodeEmail(e):
    de = ""
    k = int(e[:2], 16)

    for i in range(2, len(e)-1, 2):
        de += chr(int(e[i:i+2], 16)^k)

    return de

## Extraction process

In [ ]:
# for blogathon >= 9

data = {
    'rank': [],
    'username': [],
    'article_link': [],
    'date': [],
    'views': [],
    'creator_club_status': [],
    'article_cat': []
}

In [ ]:
# for blogathon >= 9

for i in tqdm(range(1,4)):
    xhtml = url_get_contents(f"https://datahack.analyticsvidhya.com/contest/data-science-blogathon-11/lb/{i}/").decode('utf-8')

    r = re.compile(r'<tbody>.*</tbody>', re.DOTALL)

    tableData = r.findall(xhtml)[0]

    cleanedTableData = replace_all({"<tbody>":"", "</tbody>":"", "\n": ""}, tableData).strip()[:-1]

    tableRowsList = cleanedTableData.split('</tr>')

    for i in range(len(tableRowsList)): 
        row = tableRowsList[i]
        row = row.split('</td>')[:-1]
        data['rank'].append(row[0][8:])
        data['views'].append(row[-1][4:])
        data['creator_club_status'].append(1 if "<sup>" in row[2] else 0)
        data['date'].append(row[4][4:])
        username = row[2].split('<sup>')[0].strip()[4:]
        data['username'].append(username if '[email&#160;protected]' not in username else decodeEmail(username.split('>')[0].replace('"', '').split('=')[-1]))
        data['article_link'].append(row[3].split('=')[1].replace('"', '')[:-7])
        
    time.sleep(3)

In [ ]:
# article categories extraction

for i in tqdm(range(len(data['article_link']))):
    try:
        webD = url_get_contents(data['article_link'][i]).decode('utf-8')
        cat = ','.join([i.split('>')[-1] for i in webD.split('<div class="article-categories">')[1].split('</div>')[0].replace('\n', '').split('</a>')[:-1]])
        data['article_cat'].append(cat)
    except:
        data['article_cat'].append('NA')
    time.sleep(1)

In [ ]:
# for blogathons 6,7,8

data = {
    'rank': [],
    'username': [],
    'date': [],
    'views': [],
    'creator_club_status': [],
}

In [ ]:
# for blogathons 6,7,8

for i in tqdm(range(1,6)):
    xhtml = url_get_contents(f"https://datahack.analyticsvidhya.com/contest/data-science-blogathon-8/lb/{i}/").decode('utf-8')

    r = re.compile(r'<tbody>.*</tbody>', re.DOTALL)

    tableData = r.findall(xhtml)[0]

    cleanedTableData = replace_all({"<tbody>":"", "</tbody>":"", "\n": ""}, tableData).strip()[:-1]

    tableRowsList = cleanedTableData.split('</tr>')

    for i in range(len(tableRowsList)): # len(tableRowsList)
        row = tableRowsList[i]
        row = row.split('</td>')[:-1]
        data['rank'].append(row[0][8:])
        data['views'].append(row[-1][4:])
        data['creator_club_status'].append(1 if "<sup>" in row[2] else 0)
        data['date'].append(row[4][4:])
        username = row[2].split('<sup>')[0].strip()[4:]
        data['username'].append(username if '[email&#160;protected]' not in username else decodeEmail(username.split('>')[0].replace('"', '').split('=')[-1]))
        
    time.sleep(3)

## Data transformation and saving

In [ ]:
df = pd.DataFrame(data)

In [ ]:
for i in df.columns:
    if df[i].dtype != np.int64:
        df[i] = df[i].str.replace('>', '')

In [ ]:
for i in ['rank', 'views']:
    df[i] = df[i].astype(int)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.to_csv('blogathon_8.csv')

In [ ]:
df.head()

In [ ]:
# categories counter

cat = {}
for i in df['article_cat']:
    for j in i.split(','):
        if j in cat:
            cat[j] += 1
        else:
            cat[j] = 1

In [ ]:
sorted(cat.items(), key=lambda x: x[1])[::-1]